# 🔬 Vectorius – Test Jaapd Azure OpenAI Deployment

This notebook connects to your **working Azure OpenAI deployment**:

- **Resource**: `jaapd-meryoqet-eastus2`
- **Deployment name**: `gpt-4o-mini`
- **Endpoint**: `https://jaapd-meryoqet-eastus2.openai.azure.com/`

We’ll:
1. Load credentials from `.env.local`
2. Initialize the Azure OpenAI client
3. Send a test chat completion
4. Display the response

In [7]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load variables from web/.env.local
load_dotenv(dotenv_path="../web/.env.local")

True

## 1. Grab environment variables
We expect the following entries in your `.env.local`:
```ini
AZURE_OPENAI_ENDPOINT=https://jaapd-meryoqet-eastus2.openai.azure.com/
AZURE_OPENAI_API_KEY=YOUR_KEY
AZURE_OPENAI_DEPLOYMENT=gpt-4o-mini
```

In [8]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("Endpoint:", endpoint)
print("Deployment:", deployment)
print("Key loaded:", "✔️" if key else "❌")

Endpoint: https://vectorius-openai.openai.azure.com/
Deployment: gpt-4.1-mini
Key loaded: ✔️


## 2. Initialize the Azure OpenAI client

In [9]:
client = AzureOpenAI(
    api_key=key,
    api_version="2024-12-01-preview",  # SDK version string
    azure_endpoint=endpoint
)

## 3. Send a test chat prompt
We’ll ask the model to confirm it’s alive and responding.

In [10]:
try:
    response = client.chat.completions.create(
        model=deployment,  # must match your deployment name exactly
        messages=[{"role": "user", "content": "Hello Jaapd GPT-4o-mini, can you confirm you're alive?"}]
    )
    print(response.choices[0].message)
except Exception as e:
    print("❌ Error calling endpoint:", e)

❌ Error calling endpoint: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}


## ✅ Results
If everything is configured correctly, you should see a response confirming the model is alive. If you get an error, double-check:
- Endpoint matches: `https://jaapd-meryoqet-eastus2.openai.azure.com/`
- Deployment name matches: `gpt-4o-mini`
- Key is correct in `.env.local`